In [1]:
import pandas as pd
from unidecode import unidecode

In [5]:
sac = pd.read_csv('output/bases/210727-1021-Respuestas_SAC_210708-160951-cf11_tienda-2.csv', sep=';', dtype='object')

In [12]:
tda = pd.read_csv('input/cierres_f11/tienda/210727-0940-cf11_tienda_20.csv', sep=';', dtype='object')

In [27]:
sinupc = tda[((tda['motivo']=='f4')|(tda['motivo']=='f3'))&(tda['prd_upc'].str.endswith('0000'))]

In [25]:
tda[tda.duplicated(['nfolio'])] # Verificar que no exista f11s duplicados 

,nfolio,prd_upc,qproducto,total_costo_promedio,motivo,f


In [45]:
ms = sinupc.merge(sac, on='nfolio')[['nfolio','UPC', 'F3', 'F4']]

In [46]:
ms.head()

,nfolio,UPC,F3,F4
0,1110391217,7731639115102,NaN,NaN
1,1110390708,2005223223005,32999495,NaN
2,1110035241,190198157119,NaN,NaN
3,1110056291,194850615665,32882193,NaN
4,1110189920,6934177729768,32882339,NaN


In [47]:
mts = tda.merge(ms, how='left', on='nfolio')

In [51]:
mts.loc[mts.UPC.notna(), 'prd_upc'] = mts.loc[mts.UPC.notna(), 'UPC']

In [55]:
mts.loc[(mts.F3.notna())&(mts.f.isna())]

,nfolio,prd_upc,qproducto,total_costo_promedio,motivo,f,UPC,F3,F4
537,119506264,2004915612004,1,125900,f4,NaN,2004915612004,32996784,NaN
1061,1110146794,2006452573008,1,299900,f4,NaN,2006452573008,32999365,NaN
4424,119907405,7702271818812,1,169900,f4,NaN,7702271818812,32998870,NaN
4462,119847435,2004170633004,1,184900,f4,NaN,2004170633004,32998412,NaN
4478,119821503,7702271703408,1,664900,f4,NaN,7702271703408,32843507,NaN
4522,119678688,2004933246007,1,229900,f4,NaN,2004933246007,32995718,NaN
4535,119647851,6998765471002,1,44900,f4,NaN,6998765471002,32997125,NaN
5074,1110073151,2004186488001,1,1856900,f4,NaN,2004186488001,32996654,NaN
5087,1110056451,2005866814004,1,229990,f4,NaN,2005866814004,32999136,NaN
5143,119835630,6941059648413,1,539900,f4,NaN,6941059648413,32995855,NaN


In [57]:
tda.columns

Index(['nfolio', 'prd_upc', 'qproducto', 'total_costo_promedio', 'motivo',
       'f'],
      dtype='object')

In [60]:
mtsu = mts[['nfolio', 'prd_upc', 'qproducto', 'total_costo_promedio', 'motivo',
       'f']]

## Agregar estado del f11

In [61]:
f11 = pd.read_csv('output/bases/210727-1134-Ajustes_Toma_de_Inventario-FALL-PC0GYV9W.csv', sep=';', dtype='object')

In [62]:
f11.columns

Index(['NFOLIO', 'NSECUENCIA', 'PRD_UPC', 'XSUBPROD', 'QPRODUCTO',
       'MPRECIO_VTA', 'TOTAL_MPRECIO', 'CESTADO_DT', 'DPACTADA_DT',
       'CSUCURSAL',
       ...
       'DÍAS 30 JUN', 'ANTIGÜEDAD 30 JUN', 'ESTADO ACTUAL DEL F11 - 7 JUL',
       'ESTADO 7 JUL', 'DÍAS 7 JUL', 'ANTIGÜEDAD 7 JUL', 'estado_f11',
       'ESTADO 26 JUL', 'DÍAS 26 JUL', 'ANTIGÜEDAD 26 JUL'],
      dtype='object', length=122)

In [63]:
fax= f11[['NFOLIO', 'PRD_UPC', 'estado_f11']]

In [64]:
fax

,NFOLIO,PRD_UPC,estado_f11
0,1110391088,7709577513335,Entrega total
1,1110391125,8806090658723,Entrega total
2,1110391163,17817783385,Despachado
3,1110095771,194441206470,Entrega total
4,1110095863,8806087044102,Despachado
...,...,...,...
38174,119829042,238785,Entrega total
38175,119829332,439748,Entrega total
38176,119765036,575929,Entrega total
38177,119765043,575929,Entrega total


In [68]:
save = mtsu.merge(fax, how='left', left_on=['nfolio'], right_on=['NFOLIO'])

In [76]:
save.head()

,nfolio,prd_upc,qproducto,total_costo_promedio,motivo,f,NFOLIO,PRD_UPC,estado_f11
0,1110391040,8806090523496,1,633646607,sin informacion - tienda,NaN,1110391040,8806090523496,entrega total
1,1110391071,6941059635529,1,731019324,sin informacion - tienda,NaN,1110391071,6941059635529,entrega total
2,1110391088,7709577513335,1,976060707,f3,32884210,1110391088,7709577513335,entrega total
3,1110391101,840023204012,1,804933347,sin informacion - tienda,NaN,1110391101,840023204012,entrega total
4,1110391118,8806090751967,1,1973876471,sin informacion - tienda,NaN,1110391118,8806090751967,entrega total


In [70]:
save[save.estado_f11.isna()] # Validación de estado del f11

,nfolio,prd_upc,qproducto,total_costo_promedio,motivo,f,NFOLIO,PRD_UPC,estado_f11


In [74]:
def clean_str(col):
    res = col.fillna('nan')
    res = res.apply(unidecode)
    res = res.str.replace(r'([^a-zA-Z0-9-+(). ])', '', regex=True)
    res = res.str.strip()
    res = res.str.lower()
    return res 

save['estado_f11'] = clean_str(save['estado_f11'])

In [77]:
save.to_csv(f'input/cierres_f11/tienda/210727-0940-cf11_tienda_20.csv', sep=';', decimal=',', index=False)